# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0e2417a880>
├── 'a' --> tensor([[-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513]])
└── 'x' --> <FastTreeValue 0x7f0e2417aac0>
    └── 'c' --> tensor([[-0.5386, -0.4743, -0.0267, -0.2617],
                        [ 1.0280,  0.7239, -0.5424, -1.1147],
                        [-1.1872, -0.0038, -1.9956,  0.6355]])

In [4]:
t.a

tensor([[-0.4724,  2.1265, -0.4180],
        [ 0.0832, -1.8861, -0.1513]])

In [5]:
%timeit t.a

72.1 ns ± 0.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0e2417a880>
├── 'a' --> tensor([[ 0.3149,  0.7455, -0.3079],
│                   [ 0.0750,  1.0286,  0.5498]])
└── 'x' --> <FastTreeValue 0x7f0e2417aac0>
    └── 'c' --> tensor([[-0.5386, -0.4743, -0.0267, -0.2617],
                        [ 1.0280,  0.7239, -0.5424, -1.1147],
                        [-1.1872, -0.0038, -1.9956,  0.6355]])

In [7]:
%timeit t.a = new_value

71.4 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4724,  2.1265, -0.4180],
               [ 0.0832, -1.8861, -0.1513]]),
    x: Batch(
           c: tensor([[-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355]]),
       ),
)

In [10]:
b.a

tensor([[-0.4724,  2.1265, -0.4180],
        [ 0.0832, -1.8861, -0.1513]])

In [11]:
%timeit b.a

64.7 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5740,  0.0468,  0.1646],
               [ 0.2981,  0.5067, -1.0830]]),
    x: Batch(
           c: tensor([[-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355]]),
       ),
)

In [13]:
%timeit b.a = new_value

538 ns ± 0.262 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

906 ns ± 0.859 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

149 µs ± 7.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0d499beb80>
├── 'a' --> tensor([[[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]],
│           
│                   [[-0.4724,  2.1265, -0.4180],
│                    [ 0.0832, -1.8861, -0.1513]]])
└── 'x' --> <FastTreeValue 0x7f0d499beb50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0d5c7060a0>
├── 'a' --> tensor([[-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513],
│                   [-0.4724,  2.1265, -0.4180],
│                   [ 0.0832, -1.8861, -0.1513]])
└── 'x' --> <FastTreeValue 0x7f0e2413e0a0>
    └── 'c' --> tensor([[-0.5386, -0.4743, -0.0267, -0.2617],
                        [ 1.0280,  0.7239, -0.5424, -1.1147],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 45 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.4 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5386, -0.4743, -0.0267, -0.2617],
                       [ 1.0280,  0.7239, -0.5424, -1.1147],
                       [-1.1872, -0.0038, -1.9956,  0.6355]],
              
                      [[-0.5386, -0.4743, -0.0267, -0.2617],
                       [ 1.0280,  0.7239, -0.5424, -1.1147],
                       [-1.1872, -0.0038, -1.9956,  0.6355]],
              
                      [[-0.5386, -0.4743, -0.0267, -0.2617],
                       [ 1.0280,  0.7239, -0.5424, -1.1147],
                       [-1.1872, -0.0038, -1.9956,  0.6355]],
              
                      [[-0.5386, -0.4743, -0.0267, -0.2617],
                       [ 1.0280,  0.7239, -0.5424, -1.1147],
                       [-1.1872, -0.0038, -1.9956,  0.6355]],
              
                      [[-0.5386, -0.4743, -0.0267, -0.2617],
                       [ 1.0280,  0.7239, -0.5424, -1.1147],
                       [-1.1872, -0.0038, -1.9956,  0.6355]],

In [26]:
%timeit Batch.stack(batches)

84 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355],
                      [-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355],
                      [-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355],
                      [-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355],
                      [-0.5386, -0.4743, -0.0267, -0.2617],
                      [ 1.0280,  0.7239, -0.5424, -1.1147],
                      [-1.1872, -0.0038, -1.9956,  0.6355],
                      [-0.5386, -0.4743, -0.0267, -0.2617],
                   

In [28]:
%timeit Batch.cat(batches)

175 µs ± 628 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

344 µs ± 4.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
